# Import Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
from sklearn import metrics

# DNN

## Split Data

In [ ]:
x_data = pdf.iloc[:,:len(pdf.columns)-1].to_numpy()
y_data = pdf.iloc[:,-1].to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=0)
print(x_train.shape, x_val.shape, x_test.shape)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1,1)
x_val = x_val.reshape(x_val.shape[0],-1,1)
x_test = x_test.reshape(x_test.shape[0],-1,1)

## Set Environment

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Set Model & Train

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Dropout,Input, Activation, BatchNormalization

model = Sequential()
model.add(Dense(128, activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=6, mode='min'),
             tf.keras.callbacks.ModelCheckpoint('./best1.h5', monitor='val_loss',save_best_only=True, mode='min')
            ]

model.fit(x_train, y_train, epochs=100, validation_data = (x_val, y_val),
          batch_size=32, verbose=1, callbacks=callbacks)

## Evaluate

In [ ]:
y_pred = model.predict(x_test)

y_pred = y_pred.reshape(-1)

for y in range(len(y_pred)):
    if y_pred[y] >= 0.5:
        y_pred[y] = 1
    else:
        y_pred[y] = 0
        
y_pred = y_pred.reshape(-1,1)
y_pred

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

precision = metrics.precision_score(y_test, y_pred)
print("정밀도:", precision)

recall = metrics.recall_score(y_test, y_pred)
print("재현율:", recall)

f1 = metrics.f1_score(y_test, y_pred)
print("f1 점수:", f1)